# Рекомендательные системы

### Урок 5. Поиск похожих товаров и пользователей. Гибридные рекомендательные системы

### Домашнее задание

1) Прочитать статьи про BPR, WARP loss

2) Сделать грид серч текущей модели, смотрите на метрику precision@5, считаем на тесте нашей функцией

Выполнил Соковнин ИЛ

In [3]:
# !pip install lightfm

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

from lightfm import LightFM

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

ModuleNotFoundError: No module named 'implicit'

In [ ]:
from lightfm.evaluation import precision_at_k, recall_at_k

from metrics import precision_at_k as custom_precision, recall_at_k
from utils import prefilter_items

In [ ]:
data = pd.read_csv('../data/retail_train.csv')

item_features = pd.read_csv('../data/product.csv')
user_features = pd.read_csv('../data/hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

# train test split
test_size_weeks = 3  # 3 weeks

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

In [ ]:
pwd

In [ ]:
ls

In [ ]:
ls ../data/

In [ ]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

In [ ]:
item_features.head(2)

In [ ]:
user_features.head(2)

In [ ]:
user_features['age_desc'].unique()

In [ ]:
user_features['marital_status_code'].unique()

In [ ]:
user_features['household_size_desc'].unique()

## 1. Filter items

Фильтрация товара. Оставляем 5000 самых популярных товаров.

In [ ]:
data_train.head(3)

In [ ]:
n_items_before = data_train['item_id'].nunique()

data_train_filtered = prefilter_items(data_train, take_n_popular=5000, item_features=item_features)

n_items_after = data_train_filtered['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

# 2. Prepare data set

## 2.1 Prepare csr train matrix

In [ ]:

user_item_matrix = pd.pivot_table(data_train_filtered, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(2)

## 2.2 Prepare CSR test matrix

In [ ]:
data_test = data_test[data_test['item_id'].isin(data_train['item_id'].unique())]

test_user_item_matrix = pd.pivot_table(data_test, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

test_user_item_matrix = test_user_item_matrix.astype(float) # необходимый тип матрицы для implicit

In [ ]:
test_user_item_matrix.head(2)

In [ ]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

# Cтроим матрицу преобразования
id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

## 3. Prepare user and item features

In [ ]:
user_feat = pd.DataFrame(user_item_matrix.index)
user_feat

In [ ]:
# Объединение — функция pandas. merge() соединяет строки в Dataframe на основе одного или нескольких ключей.
user_feat = user_feat.merge(user_features, on='user_id', how='left')
user_feat.set_index('user_id', inplace=True)
user_feat.head(100)

In [ ]:
user_feat.shape

In [ ]:
item_feat = pd.DataFrame(user_item_matrix.columns)
item_feat = item_feat.merge(item_features, on='item_id', how='left')
item_feat.set_index('item_id', inplace=True)

item_feat.head(2)

In [ ]:
item_feat.shape

## Encoding features

In [ ]:
# Кодирование в бинарном виде
user_feat_lightfm = pd.get_dummies(user_feat, columns=user_feat.columns.tolist())
item_feat_lightfm = pd.get_dummies(item_feat, columns=item_feat.columns.tolist())

In [ ]:
user_feat_lightfm.head(2)

## Init model

In [ ]:
model = LightFM(no_components=40,
                loss='bpr', # "logistic","bpr"
                learning_rate=0.01, 
                item_alpha=0.4,
                user_alpha=0.1, 
                random_state=42,
                k=5,
                n=15,
                max_sampled=100)

## Train

In [ ]:
model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
          sample_weight=coo_matrix(user_item_matrix),
          user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
          item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
          epochs=20, 
          num_threads=20,
          verbose=True) 

# Getting embeddings
Векторизация

## вектора по пользователям

In [ ]:
user_emb = model.get_user_representations(features=csr_matrix(user_feat_lightfm.values).tocsr())

In [ ]:
user_emb[0].shape # biases

In [ ]:
user_emb[1].shape # users vectors

In [ ]:
# user_emb

## вектора по товарам

In [ ]:
item_emb = model.get_item_representations(features=csr_matrix(item_feat_lightfm.values).tocsr())

In [ ]:
item_emb[0].shape # biases

In [ ]:
item_emb[1].shape # items vectors

In [ ]:
# item_emb

# Evaluation -> Train precision

### Оценка -> Тренировка точности

In [ ]:
# мы можем использовать встроенные метрики lightFM
train_precision = precision_at_k(model, sparse_user_item, 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()

print(f"Train precision {train_precision}")

# Predict

In [ ]:
# подготавливаемм id для юзеров и товаров в порядке пар user-item
users_ids_row = data_train_filtered['user_id'].apply(lambda x: userid_to_id[x]).values.astype(int)
items_ids_row = data_train_filtered['item_id'].apply(lambda x: itemid_to_id[x]).values.astype(int)

In [ ]:
users_ids_row[:10]

In [ ]:
items_ids_row[:10]

In [ ]:
# модель возвращает меру/скор похожести между соответствующим пользователем и товаром
predictions = model.predict(user_ids=users_ids_row,
                            item_ids=items_ids_row,
                            user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                            item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                            num_threads=10)

In [ ]:
# добавляем наш полученный скор в трейн датафрейм
data_train_filtered['score'] = predictions

In [ ]:
data_train_filtered.head()

In [ ]:
# создаем предикт датафрейм в формате списка товаров
predict_result = data_train_filtered[['user_id','item_id','score']][data_train_filtered.item_id != 999999].drop_duplicates().sort_values(by=['user_id','score'], ascending=False).groupby('user_id')['item_id']. \
            unique().reset_index()

In [ ]:
predict_result.head()

In [ ]:
# объединяем предикт и тест датасет для подсчета precision
df_result_for_metrics = result.merge(predict_result, on='user_id', how='inner')

In [ ]:
df_result_for_metrics.head()

### Test with custom precision func

In [ ]:
precision = df_result_for_metrics.apply(lambda row: custom_precision(row['item_id'], row['actual'],k=5), axis=1).mean()
print(f"Precision: {precision}")

# Links

Neural networks for RS: http://d2l.ai/chapter_recommender-systems/mf.html

LigthFM -> https://arxiv.org/pdf/1507.08439.pdf

https://making.lyst.com/lightfm/docs/home.html

# Домашнее задание

1. Прочитать статьи про BPR, WARP loss
2. Сделать грид серч текущей модели, смотрите на метрику precision@5, считаем на тесте нашей функцией

# <span style='color:blue'>Ищем оптимальные параметры (grid-search)</span>

In [ ]:
import itertools
import copy

In [ ]:
def get_predicts(users_ids_row, items_ids_row):
       
    # модель возвращает меру/скор похожести между соответствующим пользователем и товаром
    predictions = model.predict(user_ids=users_ids_row,
                                item_ids=items_ids_row,
                                user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                num_threads=10)
    # добавляем наш полученный скор в трейн датафрейм
    data_train_filtered['score'] = predictions
    # создаем предикт датафрейм в формате списка товаров
    predict_result = data_train_filtered[['user_id','item_id','score']][data_train_filtered.item_id != 999999]. \
            drop_duplicates().sort_values(by=['user_id','score'], ascending=False).groupby('user_id')['item_id']. \
            unique().reset_index()
    
    # объединяем предикт и тест датасет для подсчета precision
    # df_result_for_metrics 
    res = result.merge(predict_result, on='user_id', how='inner')
    
    return res

In [ ]:
def learning_curve(model, user_item_matrix, epochs, num_threads):
    user_item_precision = []
    
#     headers = ['epochs', 'p@k user_item_matrix']

    
    model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
      sample_weight=coo_matrix(user_item_matrix),
      user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
      item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
      # epochs=20, 
      # num_threads=20,
      verbose=True) 

    # подготавливаемм id для юзеров и товаров в порядке пар user-item
    users_ids_row = data_train_filtered['user_id'].apply(lambda x: userid_to_id[x]).values.astype(int)
    items_ids_row = data_train_filtered['item_id'].apply(lambda x: itemid_to_id[x]).values.astype(int)
    
    df_result_for_metrics = get_predicts(users_ids_row, items_ids_row)
    precision = df_result_for_metrics.apply(lambda row: custom_precision(row['item_id'], \
                                                                         row['actual'],k=5), axis=1).mean()
    user_item_precision.append(precision)

    return model, user_item_precision

In [ ]:
def grid_search_learning_curve(base_model, user_item_matrix, param_grid,
                               user_index=None, patk=5, epochs=range(2, 40, 2)):
    """
    "Inspired" (stolen) from sklearn gridsearch
    https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/model_selection/_search.py
    """
    curves = []
    keys, values = zip(*param_grid.items())
    for v in itertools.product(*values):
        params = dict(zip(keys, v))
        this_model = copy.deepcopy(base_model)
        print_line = []
        for k, v in params.items():
            setattr(this_model, k, v)
            print_line.append((k, v))

        print(' | '.join('{}: {}'.format(k, v) for (k, v) in print_line))
        
        _, user_item_patk = learning_curve(this_model, user_item_matrix)
        
        curves.append({'params': params,
                       'patk': {'user_item_matrix': user_item_patk}})
    return curves

In [ ]:
param_grid = {'no_components': [20, 40],
              'learning_rate': [0.01],
              'item_alpha': [0.4],
              'user_alpha': [0.1],              
              'k': [3, 5],
              'max_sampled': [50, 100],
             }

user_index = range(user_item_matrix.shape[0])

In [ ]:
base_model = LightFM()

In [ ]:
curves = grid_search_learning_curve(base_model, user_item_matrix,
                                    param_grid,
                                    user_index=user_index,
                                    patk=5)

user_index = range(user_item_matrix.shape[0])

In [ ]:
type(curves)

In [ ]:
curves[:5]